In [1]:
from data_processing import *
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from model import *

import numpy as np
import os

In [2]:
print(os.getcwd())
velocity_path = '../data/straight'
normalized_velocities, _ = combine_and_normalize_velocity_files(velocity_path)

c:\Users\kxfor\OneDrive\Documents\Projects\Autonomous-Driving-Project\path_tracking


In [3]:
print(np.info(normalized_velocities))

class:  ndarray
shape:  (91, 2)
strides:  (16, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x1da17c9ff10
byteorder:  little
byteswap:  False
type: float64
None


In [6]:
data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to ImageNet standards
])

In [8]:
image_folder_path = '../data/straight'
# Assuming 'normalized_velocities' and 'image_folder_path' are already defined
full_dataset = PathFollowingDataset(image_folder_path, normalized_velocities, transform=data_transform)

print(full_dataset.image_filenames[:5])
# Define the size of the training and validation sets 80/20 train/test split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Randomly split the dataset into training and validation datasets
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
print(full_dataset[1])

['image0.jpg', 'image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg']
{'image': tensor([[[ 0.0275,  0.0353,  0.0431,  ..., -0.8824, -0.8902, -0.8667],
         [ 0.0275,  0.0353,  0.0431,  ..., -0.8824, -0.8902, -0.8667],
         [ 0.0275,  0.0353,  0.0431,  ..., -0.8902, -0.8902, -0.8667],
         ...,
         [-0.5059, -0.5059, -0.5059,  ..., -0.7882, -0.7804, -0.7804],
         [-0.4980, -0.4980, -0.5059,  ..., -0.7961, -0.7882, -0.7882],
         [-0.5686, -0.5765, -0.5765,  ..., -0.8039, -0.7961, -0.7961]]]), 'velocity': tensor([0.3630, 0.3920])}


c:\Users\kxfor\OneDrive\Documents\Projects\Autonomous-Driving-Project\path_tracking\data_processing.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'image': torch.tensor(image, dtype=torch.float), 'velocity': torch.tensor(velocity, dtype=torch.float)}


In [9]:

# Define the DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [10]:
model = CNNRegressor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function
criterion = torch.nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate can be adjusted

In [11]:
def train_and_validate_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=25):
    model.train()  # Set the model to training mode initially
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Training Phase
        for i, data in enumerate(train_dataloader, 0):
            inputs = data['image'].to(device)
            targets = data['velocity'].to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 10 == 9:    # Print every 10 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 10:.4f}')
                running_loss = 0.0

        # Validation Phase
        model.eval()  # Set the model to evaluation mode for validation
        val_loss = 0.0
        with torch.no_grad():
            for data in val_dataloader:
                inputs = data['image'].to(device)
                targets = data['velocity'].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_dataloader)
        print(f'Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}')

        model.train()  # Reset to training mode for the next epoch

    print('Finished Training and Validation')


In [12]:
train_and_validate_model(model, train_loader, val_loader, criterion, optimizer)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x123808 and 800x120)